We're going to build a simple Agent system for generating cold outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:

Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.

In [19]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
from sendgrid import SendGridAPIClient
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [20]:
load_dotenv(override=True)

True

In [21]:
instructions1 = "You are a AI agent working for Anand Jain, \
a person who is actively looking for job change and very professional in nature. \
Email: anandjain2507@gmail.com. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging AI agent working for Anand Jain, \
a person who is actively looking for job change and very professional in nature. \
Email: anandjain2507@gmail.com. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy AI agent working for Anand Jain, \
a person who is actively looking for job change and very professional in nature. \
Email: anandjain2507@gmail.com. \
You write concise, to the point cold emails."

In [22]:
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
openrouter_client = AsyncOpenAI(base_url=OPENROUTER_BASE_URL, api_key=os.getenv("OPENROUTER_API_KEY"))
openrouter_model = OpenAIChatCompletionsModel(model="meta-llama/llama-3.3-8b-instruct:free", openai_client=openrouter_client)

In [23]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=os.getenv("GEMINI_API_KEY"))
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
openrouter_model=gemini_model

In [24]:
ai_agent1 = Agent(
    name="Professional AI Agent",
    instructions=instructions1,
    model=openrouter_model
)

ai_agent2 = Agent(
    name="Engaging AI Agent",
    instructions=instructions2,
    model=openrouter_model
)

ai_agent3 = Agent(
    name="Busy AI Agent",
    instructions=instructions3,
    model=openrouter_model
)

In [25]:
result = Runner.run_streamed(ai_agent1,input="Write a cold email for a company X which not responding although all the interview process is completed a month back")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Following Up - [Your Name] - [Job Title You Interviewed For]

Dear [Hiring Manager Name or Interviewer Name, if known, otherwise "Hiring Team"],

I hope this email finds you well.

I am writing to follow up on my application and interviews for the [Job Title You Interviewed For] position at [Company X]. I completed the interview process on [Date of Last Interview] and greatly enjoyed learning more about [Company X] and the [Job Title] role. I remain very interested in this opportunity and believe my skills and experience in [Mention 1-2 relevant skills/experiences] align well with the requirements discussed.

I understand that hiring decisions can take time. However, I would appreciate an update on the status of my application and the expected timeline for a decision. 

Thank you for your time and consideration. I look forward to hearing from you soon.

Sincerely,

Anand Jain
anandjain2507@gmail.com


In [26]:
message = "Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(ai_agent1, message),
        Runner.run(ai_agent2, message),
        Runner.run(ai_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output+ "\n\n")


Okay, here's a draft of a cold email, tailored for Anand Jain (anandjain2507@gmail.com), to follow up with Company X after a month of silence following the completion of the interview process:

**Subject: Following Up - [Your Name] - [Job Title] Application**

**Body:**

Dear [Hiring Manager Name or To Whom It May Concern],

I hope this email finds you well.

I am writing to follow up on my application and interviews for the [Job Title] position at Company X. My last interview was on [Date of Last Interview]. I enjoyed learning more about the role and the team during the interview process and remain very interested in the opportunity.

I understand that hiring decisions can sometimes take time, but as it has been approximately a month since my last interview, I was hoping to get an update on the timeline for a decision.

Thank you for your time and consideration. I look forward to hearing from you soon.

Sincerely,

Anand Jain
anandjain2507@gmail.com
[Your Phone Number]
[Link to your L

In [27]:
ai_picker = Agent(
    name="ai_picker",
    instructions="You are a helpful assistant that picks the best AI agent response for the cold email from the given options. \
        Imagine you are a employer for software company and pick the one you are most likely to respond to. \
        Do not give an explanation; reply with the selected email only",
    model=openrouter_model
)

In [28]:
message = "Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back"

with trace("Selection from employer people"):
    results = await asyncio.gather(
        Runner.run(ai_agent1, message),
        Runner.run(ai_agent2, message),
        Runner.run(ai_agent3, message),
    )

outputs = [result.final_output for result in results]

emails = "Cold emails:\n\n".join(outputs)

best = await Runner.run(ai_picker, emails)

print(f"Best cold email:\n{best.final_output}")

Best cold email:
Subject: Following Up - Anand Jain - [Job Title You Interviewed For]

Dear [Hiring Manager Name],

Hope you're having a productive week.

Following up on my interview process for the [Job Title] role at [Company X], completed on [Date]. Eager to understand the timeline for a decision.

Thank you for your time and consideration.

Sincerely,

Anand Jain
anandjain2507@gmail.com



Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [29]:
ai_agent1 = Agent(
    name="Professional AI Agent",
    instructions=instructions1,
    model=openrouter_model
)

ai_agent2 = Agent(
    name="Engaging AI Agent",
    instructions=instructions2,
    model=openrouter_model
)

ai_agent3 = Agent(
    name="Busy AI Agent",
    instructions=instructions3,
    model=openrouter_model
)

In [30]:
ai_agent1

Agent(name='Professional AI Agent', instructions='You are a AI agent working for Anand Jain, a person who is actively looking for job change and very professional in nature. Email: anandjain2507@gmail.com. You write professional, serious cold emails.', handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x000001CECC927FB0>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [31]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all company prospects """
    sg = SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("anandjain14314_automation.bsojl@slmail.me")  # Change to your verified sender
    to_email = To("anandjain14314@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Follow up email", content).get()
    response = sg.send(mail)
    print(response.status_code)
    print(response.body)
    print(response.headers)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [32]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all company prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CECC79C540>, strict_json_schema=True)

### And you can also convert an Agent into a tool

In [33]:
tool1 = ai_agent1.as_tool(tool_name="ai_agent1", tool_description="Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back")
tool1

FunctionTool(name='ai_agent1', description='Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CECC9F6160>, strict_json_schema=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [34]:
description = "Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back"

tool1 = ai_agent1.as_tool(tool_name="ai_agent1", tool_description=description)
tool2 = ai_agent2.as_tool(tool_name="ai_agent2", tool_description=description)
tool3 = ai_agent3.as_tool(tool_name="ai_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='ai_agent1', description='Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CECA7DCD60>, strict_json_schema=True),
 FunctionTool(name='ai_agent2', description='Write a cold email for a company X which not responding for further process/discussion although all the interview process is completed a month back', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0

## And now it's time for our Manager - our planning agent

In [35]:
instructions ="You are a manager working for Anand Jain. You use the tools given to you to generate cold emails for the invertviews you have given a month back but didn't heard back from the company. \
Last interview date 13th APR 25 and documents shared on 24th APR 25 for offer processing but no response till now. \
You never generate cold emails yourself; you always use the tools. \
You try all 3 ai_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


hr_manager = Agent(name="HR Manager", instructions=instructions, tools=tools, model=openrouter_model)

message = "Send a cold email addressed to 'Dear Recruiter'"

with trace("HR manager"):
    result = await Runner.run(hr_manager, message)
    print(result)

202
b''
Server: nginx
Date: Thu, 22 May 2025 06:03:53 GMT
Content-Length: 0
Connection: close
X-Message-Id: -3YRYFwRQeqIgQOjMm9NyA
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin


RunResult:
- Last agent: Agent(name="HR Manager", ...)
- Final output (str):
    OK. I have sent the email.
- 13 new item(s)
- 5 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
